<a href="https://colab.research.google.com/github/nani0819/dictionary/blob/main/commonvoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/tugstugi/wav2vec2-large-xlsr-53-mongolian
https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLS_R_on_Common_Voice.ipynb#scrollTo=YELVqGxMxnbG

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Apr  6 23:57:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.11.3
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer

In [3]:
%%capture
!apt install git-lfs

In [4]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

test_dataset = load_dataset("common_voice", "mn", split="test[:2%]")

processor = Wav2Vec2Processor.from_pretrained("tugstugi/wav2vec2-large-xlsr-53-mongolian")
model = Wav2Vec2ForCTC.from_pretrained("tugstugi/wav2vec2-large-xlsr-53-mongolian")

resampler = torchaudio.transforms.Resample(48_000, 16_000)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
# def speech_file_to_array_fn(batch):
#     speech_array, sampling_rate = torchaudio.load(batch["path"])
#     batch["speech"] = resampler(speech_array).squeeze().numpy()
#     return batch

# test_dataset = test_dataset.map(speech_file_to_array_fn)
# inputs = processor(test_dataset["speech"][:2], sampling_rate=16_000, return_tensors="pt", padding=True)

# with torch.no_grad():
#     logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

# predicted_ids = torch.argmax(logits, dim=-1)

# print("Prediction:", processor.batch_decode(predicted_ids))
# print("Reference:", test_dataset["sentence"][:2])

Downloading:   0%|          | 0.00/4.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/486M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

...

Downloading:   0%|          | 0.00/138 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

/usr/local/lib/python3.9/dist-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

In [5]:
from datasets import load_dataset, load_metric, Audio
common_voice_train = load_dataset("common_voice", "mn", split="train+validation")
common_voice_test = load_dataset("common_voice", "mn", split="test")

In [6]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [7]:
print(common_voice_train)

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 4020
})


In [8]:
print(common_voice_test)

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 1862
})


In [9]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,"""Сурвалжийн энэхүү мэдээнд итгэвэл, түүний """"хан"""" цол нь """"шидау хан"""" гэдгээс хураамжлагдсан нэршил болно."""
1,"Далха тахиулах нь эр хүний хийморийг сэргээх, цэргийн хүний ажил үйлсийг даатгахад уншуулдаг ном."
2,Авхай энд ирээд тогловол дээр юм болов уу?
3,За Дэндэв байгаа юм юу алзах вэ?
4,өдрийн тэмдэглэлээ нөгөө талд нь байрлуул.
5,"Вантай гүнгэрэг хэмээх номыг өнгөрсөн хүний төрлийг дээшлүүлэхэд, зам мөрийг нь хөтлөхөд айлтгуулна."
6,Сониноо алдаад солигдсон хуучин эхнэр сонин байна уу?
7,"Би Должингоос юу ч асууж зүрхлэхгүй, гагцхүү асуусанд нь хариулж явлаа."
8,Дүнсгэр жижигхэн сумын захиргаа бараг хүнгүй байлаа.
9,"Өөрсдийн санаа зөв гэдгийг баталж, радийг тэдэнд үзүүлэхийн тулд дахин дөрвөн жил ажиллажээ."


In [11]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\«\»\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [12]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [13]:
chars_to_remove_regex = 'h'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [14]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,зуслангийн байшин бол хотын чимээ хурц гэрэл замын түгжрэлээс холдон тайвшрах газар тань юм
1,тиймээс нэг шүлгийг хоёр удаа хараад л цээжээр алдаагүй уншиж чадна гэж хэлдэг байв
2,аав ийнхүү сүүлчийнхээ үгийг хоолой руугаа залгин хэлэхэд гадаа бороо шаагьсаар байлаа
3,бригадын төвд бага сургууль хүний эмнэлэгтэй тул суурин газрын хэр төрхийг нэгнээ санагдуулна
4,хатантан өргөөндөө бараалхаж энэ шөнө амарна уу
5,хүмүүст төдийгүй хөшөөнд ч бас хувь заяа байдаг биз
6,оршин байгаа олон нийтийн соёл тэдний ижилслийг тодорхойлох гол хэмжүүр болно
7,түүнийг хааны хамгийн сайн анд төдийгүй хуурай хүүд тооцдог байж
8,ийм нэвтрүүлэг бүтээгчид сонсогчдыг татаж орлогын шинэ эх сурвалж бий болгохыг зорьж байна
9,миний түүхийг бас сонсож чамаас дутуугүй зовлонт амьтан болохыг минь үзэж мэд


In [15]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [16]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [18]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'а': 1,
 'б': 2,
 'в': 3,
 'г': 4,
 'д': 5,
 'е': 6,
 'ж': 7,
 'з': 8,
 'и': 9,
 'й': 10,
 'к': 11,
 'л': 12,
 'м': 13,
 'н': 14,
 'о': 15,
 'п': 16,
 'р': 17,
 'с': 18,
 'т': 19,
 'у': 20,
 'ф': 21,
 'х': 22,
 'ц': 23,
 'ч': 24,
 'ш': 25,
 'ъ': 26,
 'ы': 27,
 'ь': 28,
 'э': 29,
 'ю': 30,
 'я': 31,
 'ё': 32,
 'ү': 33,
 'ө': 34}

In [19]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [20]:
# vocab_dict["|"] = vocab_dict[" "]
# del vocab_dict["h"]

In [21]:
# vocab_dict["[UNK]"] = len(vocab_dict)
# vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

35

In [22]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [23]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

file ./config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [25]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [26]:
# common_voice_train[0]["path"]

In [27]:
common_voice_train[0]["audio"]

{'path': 'cv-corpus-6.1-2020-12-11/mn/clips/common_voice_mn_18565371.mp3',
 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00014102,
        -0.00013202, -0.00011754], dtype=float32),
 'sampling_rate': 48000}

In [28]:
import datasets

In [29]:
common_voice_train = common_voice_train.cast_column("audio", datasets.features.audio.Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", datasets.features.audio.Audio(sampling_rate=16_000))

In [30]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

монгол бичгийн зуун дамнах шандас цаг цагийн амьсгалд өчүүхэн төдий үл саатах нигууртай ажээ


In [31]:
common_voice_train[0]["audio"]["array"]

array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
       -9.1138834e-05, -1.1053897e-04, -1.4741869e-04], dtype=float32)

In [32]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: галт тэрэгний буудалд зорчигчдын хөдөлгөөн нэн өнөр
Input array shape: (83712,)
Sampling rate: 16000


In [33]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [34]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [35]:
max_input_length_in_sec = 5.0
common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/5 [00:00<?, ?ba/s]

In [36]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [37]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [38]:
from datasets import load_dataset, load_metric, Audio

In [39]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

In [40]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [41]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "tugstugi/wav2vec2-large-xlsr-53-mongolian", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

/usr/local/lib/python3.9/dist-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [42]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='repo_name',
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=False,
)

In [43]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

Using amp fp16 backend


In [44]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 1919
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1800
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like t

Step,Training Loss,Validation Loss,Wer
400,2.263400,0.828798,0.528273
800,0.140200,0.830012,0.537155
1200,0.098600,0.845998,0.500193
1600,0.070600,0.811584,0.477906


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 1862
  Batch size = 8
Saving model checkpoint to repo_name/checkpoint-400
Configuration saved in repo_name/checkpoint-400/config.json
Model weights saved in repo_name/checkpoint-400/pytorch_model.bin
Configuration saved in repo_name/checkpoint-400/preprocessor_config.json
/usr/local/lib/python3.9/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
The followi

Step,Training Loss,Validation Loss,Wer
400,2.263400,0.828798,0.528273
800,0.140200,0.830012,0.537155
1200,0.098600,0.845998,0.500193
1600,0.070600,0.811584,0.477906




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1800, training_loss=0.5780021789338854, metrics={'train_runtime': 8476.1714, 'train_samples_per_second': 6.792, 'train_steps_per_second': 0.212, 'total_flos': 7.124680585822395e+18, 'train_loss': 0.5780021789338854, 'epoch': 30.0})

In [45]:
common_voice_test_transcription = load_dataset("common_voice", "mn", data_dir="./cv-corpus-6.1-2020-12-11", split="test")

for i in range(10):
  input_dict = processor(common_voice_test[i]["input_values"], return_tensors="pt", padding=True)

  logits = model(input_dict.input_values.to("cuda")).logits

  pred_ids = torch.argmax(logits, dim=-1)[0]

  print(f"Prediction_{i}:")
  print(processor.decode(pred_ids))

  print(f"\nReference_{i}:")
  print(common_voice_test_transcription[i]["sentence"].lower())

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

...

Prediction_6:
сайн ч мөрдөгч бас шилдэг дарга байсан юм шүү

Reference_6:
сайн ч мөрдөгч бас шилдэг дарга байсан юм шүү.
Prediction_7:


It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


эцэг эхчүүд сурагчид та бхүн доорхонаар урж захим ичээлэн хийгээрэй

Reference_7:
эцэг, эхчүүд, сурагчид та бүхэн доорх хаягаар орж цахим хичээлээ хийгээрэй.
Prediction_8:
та шин жэлийн баяраар илүү чамин бөрийн уур амьсгал оруулах гоёмсог омис бүлэлхээрэй

Reference_8:
та шинэ жилийн баяраар илүү чамин баярын уур амьсгал оруулах гоёмсог оймс бэлэглээрэй.


It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


Prediction_9:
машин ы ж овслгон тагарвал яана сайхан хувтнай баллл гэсэн үг

Reference_9:
машины донсолгоонд хагарвал яана, сайхан хувцсаа баллана л гэсэн үг.
